In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
def scrape_folder_structure(base_folder):
    results = {}
    subdirectories = ['instructpix2pix', 'kandisky', 'kandisky_bert', 'diffusion']

    for subdirectory in subdirectories:
        results[subdirectory] = []
        path_to_subdirectory = os.path.join(base_folder, subdirectory)

        for i in range(1, 101):
            numbered_subdirectory = os.path.join(path_to_subdirectory, str(i))
            if os.path.exists(numbered_subdirectory):
                file_dict = {
                    "subdirectory": subdirectory,
                    "numbered_subdirectory": i,
                    "input_path": os.path.join(numbered_subdirectory, "input.png"),
                    "groundtruth_path": os.path.join(numbered_subdirectory, "groundtruth.png"),
                    "instruction_path": os.path.join(numbered_subdirectory, "instruction.txt")
                }
                results[subdirectory].append(file_dict)

    return results

In [ ]:
def display_images(images, titles, subtitles):
    # Adjust the figure size
    fig_width = 5 * len(images)  # 5 inches per image
    fig_height = 8  # Height of the figure

    fig, axes = plt.subplots(1, len(images), figsize=(fig_width, fig_height))
    fig.suptitle(titles, fontsize=20)  # Increase font size for the title

    for ax, (img_path, subtitle) in zip(axes, zip(images, subtitles)):
        img = mpimg.imread(img_path)
        ax.imshow(img)
        ax.set_title(subtitle, fontsize=14)  # Increase font size for subtitles
        ax.axis('off')
    
    plt.show()


def main_visualization(base_folder):
    scraped_data = scrape_folder_structure(base_folder)
    
    for i in range(1, 101):
        images = []
        titles = ""
        subtitles = []
        for subdirectory in scraped_data:
            items = scraped_data[subdirectory]
            for item in items:
                if item['numbered_subdirectory'] == i:
                    if not images:
                        images.append(item['input_path'])
                        subtitles.append('Input')
                        images.append(item['groundtruth_path'])
                        subtitles.append('Groundtruth')
                        with open(item['instruction_path'], 'r') as file:
                            titles = file.readline().strip()
                    images.append(os.path.join(base_folder, subdirectory, str(i), "output.png"))
                    subtitles.append(subdirectory)

        if images:
            display_images(images, titles, subtitles)
folder = '../Outputs'
main_visualization(folder)



In [ ]:
import os
import shutil

def create_visualization_structure(base_folder, new_folder):
    scraped_data = scrape_folder_structure(base_folder)

    if not os.path.exists(new_folder):
        os.makedirs(new_folder)

    for i in range(1, 101):
        new_subdir_path = os.path.join(new_folder, f"example_{i}")
        if not os.path.exists(new_subdir_path):
            os.makedirs(new_subdir_path)

        for subdirectory in scraped_data:
            items = scraped_data[subdirectory]
            for item in items:
                if item['numbered_subdirectory'] == i:
                    # Copy input and groundtruth once per subdirectory
                    if subdirectory == 'instructpix2pix':
                        shutil.copy(item['input_path'], os.path.join(new_subdir_path, "input.png"))
                        shutil.copy(item['groundtruth_path'], os.path.join(new_subdir_path, "groundtruth.png"))

                    # Copy output
                    output_path = os.path.join(base_folder, subdirectory, str(i), "output.png")
                    if os.path.exists(output_path):
                        shutil.copy(output_path, os.path.join(new_subdir_path, f"{subdirectory}_output.png"))

folder = '../Outputs'
new_folder = '../Outputs_Visualizations'
create_visualization_structure(folder, new_folder)
